In [ ]:
## Potentially required installs: ##

# Install a pip package in the current Jupyter kernel
# Code taken from: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install PyPDF2
import sys
!{sys.executable} -m pip install pymysql
import sys
!{sys.executable} -m pip install mysql-connector



In [ ]:
## Import required packages: ##
import requests
import json
import math
import numpy
import shutil # For copying files
import time # For waiting x seconds
from datetime import date as dt # get todays date
import pandas as pd
from numpy import nan
#import PyPDF2             # For reading PDFs
#from pathlib import Path  # For writing/saving PDFs from requests
import pymysql
import mysql.connector
import os.path # Checking whether file exists in current directory

## Step 1 - Identify Companies Of Interest ##

**GetLocalActiveCompanies(api_key, location, numberOfPages=None)**  
Gives basic identifying information on all active companies in a given
location.

Input - api_key, location of interest, numberOfPages of results.  
- If numberOfPages=None then all available results will be provided.
- The current number of results per page is 20, so 2 pages gives 40 results.  


Output - Four lists, indexed equally.  
- listOfCompanyNumbers - Company number.  
- listOfCompanyNames -  Company name.  
- listOfCompanySICCodes - All of the companies SIC codes.  
- listOfCompanyAddresses - Companies full registered office address.

In [ ]:
def APIRateLimitHandler(currentCount, maxCountLimit, waitPeriod):
    currentCount = currentCount + 1
    if(currentCount == maxCountLimit): 
        time.sleep(waitPeriod + 30)
        currentCount = 0
    return currentCount

In [ ]:
def GetLocalActiveCompanies(api_key, location, currentRequestCount, maxCountLimit, waitPeriod, numberOfPages=None): 

    url = "https://api.company-information.service.gov.uk/advanced-search/companies?location="+ location +"&company_status=active"
    response = requests.get(url,auth=(api_key,''))
    currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)
    jsonSearchResult = response.text
    searchResult = json.JSONDecoder().decode(jsonSearchResult)
    
    hits = searchResult["hits"] 
    itemsPerPage = 20
    if numberOfPages==None: numberOfPages = math.ceil(hits/itemsPerPage)

    listOfCompanyNumbers = []
    listOfCompanyNames = []
    listOfCompanyAddresses = []
    listOfCompanySICCodes = []

    for page in range(0,numberOfPages):
        pageStartIndex = page * itemsPerPage
        url = "https://api.company-information.service.gov.uk/advanced-search/companies?location="+ location +"&company_status=active&start_index="+str(pageStartIndex)

        response = requests.get(url,auth=(api_key,''))
        currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)
        jsonSearchResult = response.text
        searchResult = json.JSONDecoder().decode(jsonSearchResult)
        companies = searchResult["items"]
    
        for company in companies:
            try:
                listOfCompanyNumbers.append(company["company_number"])
            except KeyError:
                listOfCompanyNumbers.append(None)
                
            try:
                listOfCompanyNames.append(company["company_name"])
            except KeyError:
                listOfCompanyNames.append(None)
            
            try:
                listOfCompanySICCodes.append(company["sic_codes"])
            except KeyError:
                listOfCompanySICCodes.append(None)
                
            try:
                addressAsDictionary = (company["registered_office_address"])
                registeredOfficeAddress = ""
                if "address_line_1" in addressAsDictionary: registeredOfficeAddress = registeredOfficeAddress + addressAsDictionary["address_line_1"]
                if "address_line_2" in addressAsDictionary: registeredOfficeAddress = registeredOfficeAddress + ", " + addressAsDictionary["address_line_2"]
                if "locality" in addressAsDictionary: registeredOfficeAddress = registeredOfficeAddress + ", " + addressAsDictionary["locality"]
                if "postal_code" in addressAsDictionary: registeredOfficeAddress = registeredOfficeAddress + ", " + addressAsDictionary["postal_code"]
                if "country" in addressAsDictionary: registeredOfficeAddress = registeredOfficeAddress + ", " + addressAsDictionary["country"]
                listOfCompanyAddresses.append(registeredOfficeAddress)
            except KeyError:
                listOfCompanyAddresses.append(None)
    
    return listOfCompanyNumbers, listOfCompanyNames, listOfCompanySICCodes, listOfCompanyAddresses, currentRequestCount

**Using GetLocalActiveCompanies:**

In [ ]:
api_key = "5ef9b7c0-60f2-4fd4-b88b-083e640f6c2a"
location = "Swansea"
currentRequestCount = 0
maxCountLimit = 599 
waitPeriod = 300
numberOfPages=2
listOfCompanyNumbers, listOfCompanyNames, listOfCompanySICCodes, listOfCompanyAddresses, currentRequestCount = GetLocalActiveCompanies(api_key, location, currentRequestCount, maxCountLimit, waitPeriod, numberOfPages)

print("\n Company numbers were: \n")
print(listOfCompanyNumbers)
print("\n Company names were: \n")
print(listOfCompanyNames)
print("\n Company SIC codes were: \n")
print(listOfCompanySICCodes)
print("\n Company addresses were: \n")
print(listOfCompanyAddresses)

print("\n The number of results was: \n")
print(len(listOfCompanyNumbers))



In [ ]:
def FormatPostcode(postcode):
    formattedPostcode = ""
    postcode = postcode.split()
    for i in range(len(postcode)):
        formattedPostcode = formattedPostcode + postcode[i] 
        if(i==0): formattedPostcode = formattedPostcode + " "
                
    return formattedPostcode

In [ ]:
def GetCompanyPostcodes(listOfCompanyAddresses):
    listOfCompanyPostcodes = []
    for i in range(len(listOfCompanyAddresses)):
        address = listOfCompanyAddresses[i]
        address = address.split(',')
        for j in range(len(address)):
            if((j>1) and (any(digit.isdigit() for digit in address[j]))): postcode = address[j]
                
        postcode = FormatPostcode(postcode)
        listOfCompanyPostcodes.append(postcode)
    
    return listOfCompanyPostcodes 

In [ ]:
def ProcessAddressSecondLine(streetAddress,location):
    doIWantIt = False
    listToCheckAgainst = ['road','street','park','Road','Street','Park','way','Way','lane','Lane']
    for i in range(len(listToCheckAgainst)):
        if((listToCheckAgainst[i] in streetAddress) and (streetAddress!=location)): doIWantIt = True
            
    return doIWantIt

In [ ]:
def GetCompanyStreetAddress(listOfCompanyAddresses, location):
    listOfCompanyStreetAddress = []
    for i in range(len(listOfCompanyAddresses)):
        address = listOfCompanyAddresses[i]
        address = address.split(',')
        streetAddress = ""
        for j in range(len(address)):
            if(j==0): streetAddress = streetAddress + address[j]
            if((j==1) and (any(digit.isdigit() for digit in address[j])) and (ProcessAddressSecondLine(address[j],location))): 
                streetAddress = streetAddress + " " + address[j]
                

        listOfCompanyStreetAddress.append(streetAddress)
    
    return listOfCompanyStreetAddress 

In [ ]:
listOfCompanyPostcodes = GetCompanyPostcodes(listOfCompanyAddresses)
print(listOfCompanyPostcodes)
print(len(listOfCompanyPostcodes))
listOfCompanyStreetAddress = GetCompanyStreetAddress(listOfCompanyAddresses, location)
print(listOfCompanyStreetAddress)
print(len(listOfCompanyStreetAddress))

## Step 2 - Profile Each Company ##

**RequestProfile(api_key, companyNumber)**  
Uses the company profile resource (within public data API) to get basic information on the date of company creation and company type. These peices of information are then extracted from the returned JSON object using the functions GetCompanyDateOfBirth and GetCompanyType.

Input - api_key, companyNumber. 
- api_key - Use your own.
- companyNumber - The unique ID number for the company of interest. These are obtained using the function GetLocalActiveCompanies.
 
Output - searchResult.  
- A JSON ojbect, which is then inspected for the required information by the associated aforementioned functions.

In [ ]:
def RequestProfile(api_key, companyNumber, currentRequestCount, maxCountLimit, waitPeriod):
    
    url = "https://api.company-information.service.gov.uk/company/{}"
    response = requests.get(url.format(companyNumber),auth=(api_key,''))
    currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)
    jsonSearchResult = response.text
    searchResult = json.JSONDecoder().decode(jsonSearchResult)

    return searchResult, currentRequestCount

In [ ]:
def GetCompanyDateOfBirth(companyProfile):
    dateOfCompanyCreation = companyProfile['date_of_creation']
    return dateOfCompanyCreation

In [ ]:
def GetCompanyType(companyProfile):
    companyType = companyProfile['type']
    return companyType

**Using RequestProfile, GetCompanyDateOfBirth and GetCompanyType**

In [ ]:
companyNumber = "12141111"
api_key = "5ef9b7c0-60f2-4fd4-b88b-083e640f6c2a"
currentRequestCount = 0
maxCountLimit = 599 
waitPeriod = 300
companyProfile, currentRequestCount = RequestProfile(api_key, companyNumber, currentRequestCount, maxCountLimit, waitPeriod)

dateOfCompanyCreation = GetCompanyDateOfBirth(companyProfile)
print("\nThe date Of Company Creation was: \n")
print(dateOfCompanyCreation)

companyType = GetCompanyType(companyProfile)
print("\nThe company type was: \n")
print(companyType)

print("\nThe profile info contains: \n")
print(companyProfile)

**GetIDsForAnyDocumentType(api_key, companyNumber, docType, numberOfDocuments=None)**  
Returns the transaction IDs for filed company documents, along with the corresponding document IDs. These can then be used with the Filing History API to download those documents for inspection and further analysis.

Input - api_key, companyNumber, docType, numberOfDocuments in your results.  
- The docType can be accounts, confirmation statement, etc. The documentation on these types can be found at: https://developer-specs.company-information.service.gov.uk/companies-house-public-data-api/resources/filinghistorylist?v=latest .
- If numberOfDocuments=None then all available results will be provided.
- The results are chronologically ordered with the most recent results first. Thus if you request 3 results of type "confirmation-statement", they will be the 3 most recent confirmation statements, i.e. those from the previous 3 years.
 


Output - A list containing the requested transaction IDs.  
- listOfTransactionIDs - Transaction IDs for the requested number of the (most recent) document type of interest.
- listOfDocumentIDs - The corresponding document IDs.

In [ ]:
def GetIDsForAnyDocumentType(api_key, companyNumber, docType, currentRequestCount, maxCountLimit, waitPeriod, numberOfDocuments=None): 

    url = "https://api.company-information.service.gov.uk/company/" + companyNumber + "/filing-history"

    response = requests.get(url,auth=(api_key,''))
    currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)
    jsonSearchResult = response.text
    searchResult = json.JSONDecoder().decode(jsonSearchResult)
   
    if numberOfDocuments==None: numberOfDocuments = searchResult["total_count"] 
    numberOfDocumentsAvailable = searchResult["total_count"] 
    itemsPerPage = searchResult["items_per_page"] 
    numberOfPages = math.ceil(numberOfDocumentsAvailable/itemsPerPage)
    
    listOfTransactionIDs = []
    listOfDocumentIDs = []
    listOfDocumentDates = []
    numberOfResultsCounter = 0
    
    for page in range(0,numberOfPages):
        pageStartIndex = page * itemsPerPage
        url = "https://api.company-information.service.gov.uk/company/" +companyNumber+ "/filing-history?start_index="+str(pageStartIndex)

        response = requests.get(url,auth=(api_key,''))
        currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)
        jsonSearchResult = response.text
        searchResult = json.JSONDecoder().decode(jsonSearchResult)
        documents = searchResult["items"]
    
        for document in documents:
            if (document["category"]==docType):
                
                listOfTransactionIDs.append(document["transaction_id"])                
                listOfDocumentDates.append(document["action_date"])
                
                docIDDict = document["links"] 
                urlDecompositionList = docIDDict["document_metadata"].split("/")
                listOfDocumentIDs.append(urlDecompositionList[-1])
               
                numberOfResultsCounter = numberOfResultsCounter + 1
                if (numberOfResultsCounter==numberOfDocuments): break
                    
        if (numberOfResultsCounter==numberOfDocuments): break 
        
    return listOfTransactionIDs, listOfDocumentIDs, listOfDocumentDates, currentRequestCount


**Using GetIDsForAnyDocumentType:**

In [ ]:
api_key = "5ef9b7c0-60f2-4fd4-b88b-083e640f6c2a"
companyNumber = "07804652"
docType = "accounts"
numberOfDocuments = 2
currentRequestCount = 0
maxCountLimit = 599 
waitPeriod = 300

listOfTransactionIDs, listOfDocumentIDs, listOfDocumentDates, currentRequestCount= GetIDsForAnyDocumentType(api_key, companyNumber, docType, currentRequestCount, maxCountLimit, waitPeriod, numberOfDocuments) 

print("\n The transaction IDs were: \n")
print(listOfTransactionIDs)
print(len(listOfTransactionIDs))

print("\n The document IDs were: \n")
print(listOfDocumentIDs)
print(len(listOfDocumentIDs))

print("\n The document dates were: \n")
print(listOfDocumentDates)
print(len(listOfDocumentDates))

**GetXMLFile(api_key, documentID, documentName)**  
Requests and saves the document associated with documentID into a file named documentName.

Input - api_key, documentID, documentName. 
- api_key - Use your own. 
- documentID - The document ID for the document of interest, obtained using the function GetIDsForAnyDocumentType.

Output - None 
- The file is saved into the same directory as the code is running from, i.e. the current working directory.

In [ ]:
def GetXMLFile(api_key, documentID, documentName, currentRequestCount, maxCountLimit, waitPeriod):
    url = "https://document-api.company-information.service.gov.uk/document/"+documentID+"/content"
    requestHeaders = {'Accept': 'application/xhtml+xml'}
    response = requests.get(url,auth=(api_key,''),headers=requestHeaders)
    currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)

    with open(documentName, 'wb') as f:
        f.write(response.content)
    return currentRequestCount

In [ ]:
def GetPDFFile(api_key, documentID, documentName, currentRequestCount, maxCountLimit, waitPeriod):
    url = "https://document-api.company-information.service.gov.uk/document/"+documentID+"/content"
    requestHeaders = {'Accept': 'application/pdf'}
    response = requests.get(url,auth=(api_key,''),headers=requestHeaders)
    currentRequestCount = APIRateLimitHandler(currentRequestCount, maxCountLimit, waitPeriod)

    with open(documentName, 'wb') as f:
        f.write(response.content)
    return currentRequestCount

**Using GetXMLFile:**  
(Using the document ID to get the document file from the filing history API.)

In [ ]:
api_key = "5ef9b7c0-60f2-4fd4-b88b-083e640f6c2a"
documentID = listOfDocumentIDs[0]
documentName = "Accounts.txt"
currentRequestCount = 0
maxCountLimit = 599 
waitPeriod = 300

currentRequestCount = GetXMLFile(api_key, documentID, documentName, currentRequestCount, maxCountLimit, waitPeriod)

**GetStringFromFile(filePath, targetString1, targetString2)**  
Returns the string of interest within the specified file. Specify the file, specify the point where the data is stored within the file, i.e. "cash in bank", and the subsequent line to then extract.

Input - filePath, targetString1[], targetString2.  
- filePath - the file of interest, e.g. Accounts.txt (XML file)
- targetString1[] - the part of the text from which to begin the specific seach relating to targetString2. e.g. targetString1 = ">Cash at bank and in hand". At this point the function will then search for the first line containing an instance of targetString2 and then returns that line.
- targetString1 is an array and takes several search terms, since different size companies use different formats, such as with accounts, and so different search terms will be required. It will try each of them in turn.
- targetString2 - the pattern to search for the first instance of, starting from the location of the first instance of the pattern spcified by targetString1.
 


Output - targetLine.  
- targetLine - The first line of text from the file that matches both the patterns specifies by targetString1 and targetString2.
- This string can then be processed to extract the information of interest.  


- This function is for general purpose, whenever a single line is to be extracted from a text file and can be accurately specified by the above method of using targetString1 and targetString2.

In [ ]:
def GetStringFromFile(filePath, targetString1, targetString2):
    foundLine = False
    foundSection = False
    tryNextQuery = True
    with open(filePath, 'r',encoding='utf-8') as file:
        lines = file.readlines()
        headerText = lines[0]
        headerText = headerText.split(' ')
        headerText = headerText[0]
        targetLine = "No Section Found"
        if(headerText!="<?xml"):
            targetLine = "No File"
            tryNextQuery = False
        targetCounter = 0
        while (tryNextQuery==True):
            for line in lines:
                if ((line.find(targetString1[targetCounter])!=-1) and foundSection==False):
                    foundSection = True
                    tryNextQuery = False
                    linesTried = 0
                    currentLine = lines.index(line)
                    while (foundLine==False and linesTried<10):     
                        if (targetString2 in lines[currentLine]):
                            foundLine = True
                            targetLine = lines[currentLine]
                        else:
                            currentLine = currentLine + 1
                            linesTried = linesTried + 1
                    if(linesTried==10): 
                        targetLine = "No Line Found"
                        if(targetCounter<len(targetString1)):
                            tryNextQuery = True
                            foundSection = False
                            
            targetCounter = targetCounter + 1 
    
    return targetLine

**ProcessLineCashAtBank(targetLine)**  
Returns the financial data for the cash in bank and in hand from the input string.

Input - targetLine.  
- targetLine - The string containing the financial information, obtained using the function GetStringFromFile.
 


Output - targetValue.  
- targetValue - The financial data for cash in bank and in hand, returned as an integer in units of GBP.

In [ ]:
def ProcessLineCashAtBank(targetLine):
    if(targetLine=="No File" or targetLine=="No Line Found" or targetLine=="No Section Found"):
        targetValue = targetLine
    else:
        targetValueList = targetLine.split(">")
        if(targetValueList[-1]=="\n"):
            targetValueList.remove("\n")
        if("</div" in targetValueList[-1]):
            del targetValueList[-1]
        targetValueList = targetValueList[-1].split("<")
        targetValueList = targetValueList[0].split(",")

        targetValue = ""
        for i in range(0,len(targetValueList)):
            targetValue = targetValue + str(targetValueList[i])
        targetValue = int(targetValue)
        
    return targetValue

**Using GetStringFromFile and ProcessLineCashAtBank**

In [ ]:
filePath = "Accounts.txt"
targetString1 = [">Cash at bank and in hand<",">CAPITAL AND RESERVES<", "reserves"]
targetString2 = "</ix:nonFraction>" 

targetLine = GetStringFromFile(filePath, targetString1, targetString2)
targetValue = ProcessLineCashAtBank(targetLine)   

print("\nThe cash in bank and in hand value in GBP is: \n")
print(targetValue)
print(type(targetValue))

## Step 3 - Main Function ##

**SaveCurrentCompanyErrorLog & DeleteCurrentCompanyErrorLog exist for debugging purposes:**

In [ ]:
def SaveCurrentCompanyErrorLog(companyNumber):
    errorLogName = "errorLog_companyNumbers.txt"
    if os.path.exists(errorLogName):
        with open(errorLogName, 'a+', encoding='utf-8') as f:
            f.seek(0) 
            data = f.read(100) 
            if len(data) > 0: f.write("\n")
            f.write(str(companyNumber))
    else:
        with open(errorLogName, 'w', encoding='utf-8') as f:
            f.write(str(companyNumber))
            
    return  

In [ ]:
def DeleteCurrentCompanyErrorLog():
    errorLogName = "errorLog_companyNumbers.txt"
    if os.path.exists(errorLogName):
        os.remove(errorLogName)
            
    return

**Using SaveCurrentCompanyErrorLog & DeleteCurrentCompanyErrorLog:**

In [ ]:
companyNumbers = [111,222,333,444,555]
DeleteCurrentCompanyErrorLog()
for i in range(len(companyNumbers)):
    companyNumber = companyNumbers[i]
    print(companyNumber)
    SaveCurrentCompanyErrorLog(companyNumber)

In [ ]:
def FormatStringRemoveEndSpaces(string):
    formattedString = ""
    string = string.split()

    for i in range(len(string)):
        if(i>0): formattedString = formattedString + " " + string[i] 
        else: formattedString = formattedString + string[i]
            
    return formattedString

In [ ]:
string = "   Hello  bob,, how' are'' you, , today?  "
formattedString = FormatStringRemoveEndSpaces(string)    
print(formattedString)

In [ ]:
def IsCharacterANumber(char):
    try:
        char = int(char)
        isThisCharacterANumber = True
    except ValueError:
        isThisCharacterANumber = False
    return isThisCharacterANumber

In [ ]:
def ContainsPostcode(text):
    postcodeResult = False
    for i in range(len(text)):
        result = IsCharacterANumber(text[i])
        if(result==True): postcodeResult = True
    return postcodeResult

In [ ]:
text = "Wales SA2 9DF"
text = FormatStringRemoveEndSpaces(text)
postcodeResult = ContainsPostcode(text)
if(postcodeResult==True): print("\nThis is the postcode.")
else: print("\nThis is NOT the postcode.")

In [ ]:
def FormatCaseOfName(name):
    name = name.split()
    
    for i in range(len(name)):
        name[i] = name[i].capitalize()
        
    formattedName = ""
    for i in range(len(name)):
        if(i==0): formattedName = formattedName + name[i]
        else: formattedName = formattedName + " " + name[i]
    
    return formattedName

In [ ]:
name = "milford haVen"
formattedName = FormatCaseOfName(name)
print(formattedName)

In [ ]:
def IsThisCompanyReallyFromElsewhere(nameToCheck, location):

    location = FormatStringRemoveEndSpaces(location)
    nameToCheck = nameToCheck.split(',')

    shouldIDelete = False
    previousEntryContainedLocation = False
    thisEntryContainedLocation = False
    lenghOfPreviousEntry = 0
    
    for i in range(len(nameToCheck)):
        text = FormatStringRemoveEndSpaces(nameToCheck[i])
        lenghOfThisEntry = len(text)
        if((location in text) or (location.lower() in text) or (FormatCaseOfName(location) in text)): 
            shouldIDelete = True
            thisEntryContainedLocation = True
        else: thisEntryContainedLocation = False
        
        if((previousEntryContainedLocation == True) and (ContainsPostcode(text)) and (lenghOfPreviousEntry == len(location))): 
            shouldIDelete = False
        previousEntryContainedLocation = thisEntryContainedLocation   
        lenghOfPreviousEntry = lenghOfThisEntry

    return shouldIDelete

In [ ]:
location = "Liverpool"
nameToCheck = "place, liverpool way, Swansea, Wales SA2 9DF"
result = IsThisCompanyReallyFromElsewhere(nameToCheck, location)
print("\nFor location '" + location + "' the address: '" + nameToCheck + "' returns:")
print(result)

location = "Liverpool"
nameToCheck = "place, liverpool way, Wales SA2 9DF"
result = IsThisCompanyReallyFromElsewhere(nameToCheck, location)
print("\nFor location '" + location + "' the address: '" + nameToCheck + "' returns:")
print(result)

location = "Liverpool"
nameToCheck = "place, liverpool way, liverpool, swansea, Wales SA2 9DF"
result = IsThisCompanyReallyFromElsewhere(nameToCheck, location)
print("\nFor location '" + location + "' the address: '" + nameToCheck + "' returns:")
print(result)

location = "Liverpool"
nameToCheck = "place, liverpool, liverpool, Wales SA2 9DF"
result = IsThisCompanyReallyFromElsewhere(nameToCheck, location)
print("\nFor location '" + location + "' the address: '" + nameToCheck + "' returns:")
print(result)

In [ ]:
def CheckCompanyAddressListForIncorrectEntries(listOfCompanyAddresses, location):
    badIndexes = []
    for i in range(len(listOfCompanyAddresses)):
        shouldIDelete = IsThisCompanyReallyFromElsewhere(listOfCompanyAddresses[i], location)
        if(shouldIDelete==True): badIndexes.append(i)
    
    return badIndexes

In [ ]:
listOfCompanyAddresses = ["place, swansea, sa2 9df", "another place, swansea cove, liverpool, l8 9df", "some, swansea, Leeds, L0 IH8", "yes, swansea-ov, England LP 3UI", "3ty, Swansea, Liverpool, Swansea, Wales SA2 0AH"]
location = "Swansea"
badIndexes =  CheckCompanyAddressListForIncorrectEntries(listOfCompanyAddresses, location)
print("\n The bad index list is:")
print(badIndexes)

In [ ]:
def RemoveListEntries(listToEdit, listOfBadIndex):
    
    for i in range(len(listOfBadIndex)):
        if(i==0): indexToRemove = listOfBadIndex[i]
        else: indexToRemove = listOfBadIndex[i] - i
        listToEdit.pop(indexToRemove)
    
    return listToEdit

In [ ]:
listOfCompanyAddresses = RemoveListEntries(listOfCompanyAddresses, badIndexes)
print(listOfCompanyAddresses)

In [ ]:
def SaveErrorLogForIncorrectEntries(listOfCompanyNumbers, listOfCompanyAddresses, badIndexes, location):
    
    errorLogName = "errorLogForIncorrectEntries.txt"
    with open(errorLogName, "a+", encoding='utf-8') as f:
        f.seek(0) 
        data = f.read(100) 
        if len(data) > 0: f.write("\n\n")
            
        stringToWrite = "\nThe following companies were deemed not appropriate results for the location in question: "+location+"\n"
        f.write(stringToWrite)
        
        for i in range(len(badIndexes)):
            stringToWrite = "\nCompany number - " + str(listOfCompanyNumbers[badIndexes[i]]) + " : Address - " + listOfCompanyAddresses[badIndexes[i]]
            f.write(stringToWrite)
    
    return

In [ ]:
def GetSummaryData(api_key, location, saveBadFiles, currentRequestCount, maxCountLimit, waitPeriod, numberOfDocuments, numberOfPages=None):
    
    location_city = location.split(',')[0]
    errorLogName = "errorLog"+location_city+".txt"
    with open(errorLogName, 'w', encoding='utf-8') as f:
        f.write('The following companies files were not correctly read:')
    
    listOfIncorporationDates = []
    listOfCompanyType = []
    listOfCompanyCash = []
    listOfDocumentDates = []
    
    NoneList = []
    for i in range(numberOfDocuments):
        NoneList.append(None)
    
    listOfCompanyNumbers, listOfCompanyNames, listOfCompanySICCodes, listOfCompanyAddresses, currentRequestCount = GetLocalActiveCompanies(api_key, location, currentRequestCount, maxCountLimit, waitPeriod, numberOfPages)
    
    badIndexes = CheckCompanyAddressListForIncorrectEntries(listOfCompanyAddresses, location_city)
    SaveErrorLogForIncorrectEntries(listOfCompanyNumbers, listOfCompanyAddresses, badIndexes, location_city)
    listOfCompanyNumbers = RemoveListEntries(listOfCompanyNumbers, badIndexes)
    listOfCompanyNames = RemoveListEntries(listOfCompanyNames, badIndexes)
    listOfCompanySICCodes = RemoveListEntries(listOfCompanySICCodes, badIndexes)
    listOfCompanyAddresses = RemoveListEntries(listOfCompanyAddresses, badIndexes)
    
    listOfCompanyPostcodes = GetCompanyPostcodes(listOfCompanyAddresses)
    listOfCompanyStreetAddress = GetCompanyStreetAddress(listOfCompanyAddresses, location_city)
    numberOfErrors = 0
    
    ### For Debugging ###
    #DeleteCurrentCompanyErrorLog()
    ### ------------- ###
    
    for companyNumber in listOfCompanyNumbers:
        
        ### For Debugging ###
        #SaveCurrentCompanyErrorLog(companyNumber)
        ### ------------- ###
        
        targetString1 = [">Cash at bank and in hand<",">CAPITAL AND RESERVES<","Reserves","Capital and reserves","Cash at bank", "Net assets", "SHAREHOLDERS' FUNDS"]
        targetString2 = "</ix:nonFraction>"
        targetValue = 0
        
        try:
            companyProfile, currentRequestCount = RequestProfile(api_key, companyNumber, currentRequestCount, maxCountLimit, waitPeriod)
        except:
            targetValue = None
            numberOfErrors = numberOfErrors + 1
            with open(errorLogName, "a+", encoding='utf-8') as file_object:
                file_object.seek(0) 
                data = file_object.read(100) 
                if len(data) > 0: file_object.write("\n\n")
                stringToWrite = "Company " + companyNumber +" could not be read. Failed in function RequestProfile - A profile could not be obtained for this company."
                file_object.write(stringToWrite)   
                
        if(targetValue!=None):   
            try:
                dateOfCompanyCreation = None
                dateOfCompanyCreation = GetCompanyDateOfBirth(companyProfile)
                listOfIncorporationDates.append(dateOfCompanyCreation)
            except KeyError:
                targetValue = None
                listOfIncorporationDates.append(dateOfCompanyCreation)
                numberOfErrors = numberOfErrors + 1
                with open(errorLogName, "a+", encoding='utf-8') as file_object:
                    file_object.seek(0) 
                    data = file_object.read(100) 
                    if len(data) > 0: file_object.write("\n\n")
                    stringToWrite = "Company " + companyNumber +" could not be read. KeyError - Failed in function GetCompanyDateOfBirth - This record does not contain key date_of_creation."
                    file_object.write(stringToWrite) 
            try:       
                companyType = None
                companyType = GetCompanyType(companyProfile)
                listOfCompanyType.append(companyType)
            except KeyError:
                targetValue = None
                listOfCompanyType.append(companyType)
                numberOfErrors = numberOfErrors + 1
                with open(errorLogName, "a+", encoding='utf-8') as file_object:
                    file_object.seek(0) 
                    data = file_object.read(100) 
                    if len(data) > 0: file_object.write("\n\n")
                    stringToWrite = "Company " + companyNumber +" could not be read. KeyError - Failed in function GetCompanyType - This record does not contain key Type."
                    file_object.write(stringToWrite) 
        
        docType = "accounts"
        
        if(targetValue!=None):
            try:         
                listOfTransactionIDs, listOfDocumentIDs, listOfDocumentDatesForSingleCompany, currentRequestCount = GetIDsForAnyDocumentType(api_key, companyNumber, docType, currentRequestCount, maxCountLimit, waitPeriod, numberOfDocuments) 
                
                if(len(listOfTransactionIDs)<numberOfDocuments):
                    diff = numberOfDocuments - len(listOfTransactionIDs)
                    for i in range(diff):
                        listOfTransactionIDs.append("None") 
 
                if(len(listOfDocumentIDs)<numberOfDocuments):
                    diff = numberOfDocuments - len(listOfDocumentIDs)
                    for i in range(diff):
                        listOfDocumentIDs.append("None") 

                if(len(listOfDocumentDatesForSingleCompany)<numberOfDocuments):
                    diff = numberOfDocuments - len(listOfDocumentDatesForSingleCompany)
                    for i in range(diff):
                        listOfDocumentDatesForSingleCompany.append("None")
                        
                listOfDocumentDates.append(listOfDocumentDatesForSingleCompany)    

            except:
                targetValue = None
                listOfDocumentDates.append(NoneList)
                numberOfErrors = numberOfErrors + 1
                with open(errorLogName, "a+", encoding='utf-8') as file_object:
                    file_object.seek(0) 
                    data = file_object.read(100) 
                    if len(data) > 0: file_object.write("\n\n")
                    stringToWrite = "Company " + companyNumber +" could not be read. KeyError - Failed in function GetIDsForAnyDocumentType - Document IDs could not be obtained."
                    file_object.write(stringToWrite) 
                    

        if(targetValue!=None): 
            listOfCashForSingleCompany = []
            for i in range(numberOfDocuments):
                fileName = "Accounts_" + listOfTransactionIDs[i]
                fileExtension = ".txt"
                filePath = fileName + fileExtension
                targetValue = 0

                try:
                    documentID = listOfDocumentIDs[i]
                    currentRequestCount = GetXMLFile(api_key, documentID, filePath, currentRequestCount, maxCountLimit, waitPeriod)
                    currentRequestCount = GetPDFFile(api_key, documentID, fileName+".pdf", currentRequestCount, maxCountLimit, waitPeriod)
                except IndexError:
                    targetValue = None
                    numberOfErrors = numberOfErrors + 1
                    with open(errorLogName, "a+", encoding='utf-8') as file_object:
                        file_object.seek(0) 
                        data = file_object.read(100) 
                        if len(data) > 0: file_object.write("\n\n")
                        stringToWrite = "Company " + companyNumber +" could not be read. IndexError - Failed in function GetSummaryData - Never obtained any document IDs."
                        file_object.write(stringToWrite) 
                except:
                    targetValue = None
                    numberOfErrors = numberOfErrors + 1
                    with open(errorLogName, "a+", encoding='utf-8') as file_object:
                        file_object.seek(0) 
                        data = file_object.read(100) 
                        if len(data) > 0: file_object.write("\n\n")
                        stringToWrite = "Company " + companyNumber +" could not be read. Failed when trying to get Accounts files."
                        file_object.write(stringToWrite) 
                
        
                if(targetValue!=None):
                    try:
                        targetLine = GetStringFromFile(filePath, targetString1, targetString2)
        
                        if(targetLine=="No File"):
                            targetValue = None
                            numberOfErrors = numberOfErrors + 1
                            with open(errorLogName, "a+", encoding='utf-8') as file_object:
                                file_object.seek(0) 
                                data = file_object.read(100) 
                                if len(data) > 0: file_object.write("\n\n")
                                stringToWrite = "Company " + companyNumber +" could not be read. No File - no valid XML file returned or wrong file type."
                                file_object.write(stringToWrite) 
        
                        if(targetLine=="No Section Found"):
                            targetValue = None
                            numberOfErrors = numberOfErrors + 1
                            with open(errorLogName, "a+", encoding='utf-8') as file_object:
                                file_object.seek(0) 
                                data = file_object.read(100) 
                                if len(data) > 0: file_object.write("\n\n")
                                stringToWrite = "Company " + companyNumber +" could not be read. No Section Found matching any pattern set by targetString1."
                                file_object.write(stringToWrite) 
                
                        if(targetLine=="No Line Found"):
                            targetValue = None
                            numberOfErrors = numberOfErrors + 1
                            with open(errorLogName, "a+", encoding='utf-8') as file_object:
                                file_object.seek(0) 
                                data = file_object.read(100) 
                                if len(data) > 0: file_object.write("\n\n")
                                stringToWrite = "Company " + companyNumber +" could not be read. No Line Found matching pattern set by targetString2 below the section of interest."
                                file_object.write(stringToWrite) 
            
                    except IndexError:
                        targetValue = None
                        numberOfErrors = numberOfErrors + 1
                        with open(errorLogName, "a+", encoding='utf-8') as file_object:
                            file_object.seek(0) 
                            data = file_object.read(100) 
                            if len(data) > 0: file_object.write("\n\n")
                            stringToWrite = "Company " + companyNumber +" could not be read. IndexError - Failed in function GetStringFromFile, i.e. never found an appropriate string."
                            file_object.write(stringToWrite) 
      
        
                if(targetValue!=None):
                    try:
                        targetValue = ProcessLineCashAtBank(targetLine)   
                    except ValueError:
                        targetValue = None
                        numberOfErrors = numberOfErrors + 1
                        with open(errorLogName, "a+", encoding='utf-8') as file_object:
                            file_object.seek(0)
                            data = file_object.read(100) 
                            if len(data) > 0: file_object.write("\n\n")
                            stringToWrite = "Company " + companyNumber +" could not be read. ValueError - Failed in function ProcessLineCashAtBank, i.e. likely the wrong string."
                            file_object.write(stringToWrite) 
                            file_object.write("\n") 
                            file_object.write(targetLine) 
    
                listOfCashForSingleCompany.append(targetValue)
        
                if(targetValue==None and saveBadFiles==True):
                    copyOfGuiltyFile = fileName + "_" + companyNumber + fileExtension
                    with open(filePath,'r', encoding='utf-8') as firstfile, open(copyOfGuiltyFile,'w', encoding='utf-8') as secondfile:
                        for line in firstfile:
                            secondfile.write(line)
                    copyOfGuiltyFile = fileName + "_" + companyNumber + ".pdf"
                    shutil.copyfile(fileName+".pdf", copyOfGuiltyFile)
                    
                if os.path.exists(filePath):
                    os.remove(filePath)
                if os.path.exists(fileName+".pdf"):
                    os.remove(fileName+".pdf")
                    
            listOfCompanyCash.append(listOfCashForSingleCompany)
        else: 
            listOfCompanyCash.append(NoneList)

    with open(errorLogName, "a+", encoding='utf-8') as file_object:
        file_object.seek(0)
        data = file_object.read(100) 
        if len(data) > 0: file_object.write("\n\n")
        errorRate = (numberOfErrors/len(listOfCompanyNumbers)) * 100
        stringToWrite = "Total number of documents that failed to be read was: \n" + str(numberOfErrors) + " out of " + str(len(listOfCompanyNumbers)) +" - error rate of " +str(errorRate)+ " %."
        file_object.write(stringToWrite)
    
    return listOfCompanyNumbers, listOfCompanyNames, listOfCompanySICCodes, listOfCompanyPostcodes, listOfCompanyStreetAddress, listOfIncorporationDates, listOfCompanyType, listOfCompanyCash, listOfDocumentDates, currentRequestCount

**Using GetSummaryData:**

In [ ]:

numberOfPages = 1
numberOfDocuments = 1
saveBadFiles = False

##---------------------##
##---------------------##

api_key = "5ef9b7c0-60f2-4fd4-b88b-083e640f6c2a"
currentRequestCount = 0
maxCountLimit = 599
waitPeriod = 300

currentRequestCount = 0
location = "Southport, England"     
listOfCompanyNumbers, listOfCompanyNames, listOfCompanySICCodes, listOfCompanyPostcodes, listOfCompanyStreetAddress, listOfIncorporationDates, listOfCompanyType, listOfCompanyCash, listOfDocumentDates, currentRequestCount = GetSummaryData(api_key, location, saveBadFiles, currentRequestCount, maxCountLimit, waitPeriod, numberOfDocuments, numberOfPages)
print("Company names:")
print(listOfCompanyNames)
print("\nCompany numbers:")
print(listOfCompanyNumbers)
print("\nCompany SIC codes:")
print(listOfCompanySICCodes)
print("\nCompany postcodes:")
print(listOfCompanyPostcodes)
print("\nCompany street addresses:")
print(listOfCompanyStreetAddress)
print("\nCompany incorporation dates:")
print(listOfIncorporationDates)
print("\nCompany type:")
print(listOfCompanyType)
print("\nCompany cash:")
print(listOfCompanyCash)
print("\nCompany document dates:")
print(listOfDocumentDates)